In [ ]:
import pandas as pd
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm

In [ ]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# lastfm
voy a coger todos los artistas top de la página de klastfm y voy a sacar los top géneros de dichos artistas. Para ello, se obtienen los datos con los métodos `'chart.gettopartists'` y `'artist.getTopTags'`

aquí está la docu, para más información: https://www.last.fm/api#getting-started

In [ ]:
load_dotenv()

True

In [ ]:
api=os.getenv("lastfm-id")
username=os.getenv("lastfm-user")

In [ ]:
headers = {
    'user-agent': username
}

params = {
    'api_key': api,
    'method': 'chart.gettopartists',
    'format': 'json'
}

res = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
res.status_code ## si 200 -> éxito

200

In [ ]:
res.json()

{'artists': {'artist': [{'name': 'The Weeknd',
    'playcount': '381920388',
    'listeners': '2952469',
    'mbid': 'c8b03190-306c-4120-bb0b-6f2ebfc06ea9',
    'url': 'https://www.last.fm/music/The+Weeknd',
    'streamable': '0',
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'mega'}]},
   {'name': 'Taylor Swift',
    'playcount': '1060979638',
    'listeners': '3675966',
    'mbid': '20244d07-534f-4eff-b4d4-930878889970',


me creo una función para fijar los parámetros fijos, y que el `method` entre como variable.

In [ ]:
## me voy a hacer una función con los datos de la petición
def getLastFMData(params):
    headers = {'user-agent': username} ## defino mi usuario
    url = 'https://ws.audioscrobbler.com/2.0/' ##url a la que voy a contactar

    ## configuro los params para hacer la petición
    params['api_key'] = api
    params['format'] = 'json'

    res = requests.get(url, headers=headers, params=params)
    return res

In [ ]:
## voy a probar que la petición funciona
topArtists = getLastFMData({
    'method': 'chart.gettopartists'
})
topArtists.status_code

200

In [ ]:
topArtists.json() ## me da un cacharro de tipo json, al que ya me enfrentaré luego

{'artists': {'artist': [{'name': 'The Weeknd',
    'playcount': '381920388',
    'listeners': '2952469',
    'mbid': 'c8b03190-306c-4120-bb0b-6f2ebfc06ea9',
    'url': 'https://www.last.fm/music/The+Weeknd',
    'streamable': '0',
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'mega'}]},
   {'name': 'Taylor Swift',
    'playcount': '1060979638',
    'listeners': '3675966',
    'mbid': '20244d07-534f-4eff-b4d4-930878889970',


para obtener la info concreta de los artistas, necesito saber la página de la que quiero obtener la info y el número de resultados que quiero obtener por página. https://www.last.fm/api/show/chart.getTopArtists

el número de páginas no sé cuanto es, pero el límite de artistas por páginas es 50

In [ ]:
int(topArtists.json()['artists']['@attr']['totalPages'])

88556

In [ ]:
## voy a intentar obtener la info de los artistas por cada una de las páginas
tqdm.pandas()

responses = [] ## almacenaré cada una de las peticiones por página en una lista

page_inicio = 1
page_final = int(topArtists.json()['artists']['@attr']['totalPages']) #sé el número máx de páginas de la consulta que hice anteriormente
page_limit = int(topArtists.json()['artists']['@attr']['perPage']) #sé el número de artistas que tengo por página

for page in tqdm(range(page_inicio, page_final)):
    if page <= page_final:
        ## cambio los parámetros de entrada 
        params = {
            'method': 'chart.gettopartists',
            'limit': page_limit, 
            'page': page
        }
        
        ## hago la petición con los nuevos parámetros
        res = getLastFMData(params)

        ## puedo sacar la página por la que voy
        page = int(res.json()['artists']['@attr']['page'])

        ## añado las respuesta a la lista
        responses.append(res)

        ## le sumo uno a la página para que avance
        page += 1

 71%|███████   | 62798/88555 [4:31:53<1:51:31,  3.85it/s] 


ConnectionError: HTTPSConnectionPool(host='ws.audioscrobbler.com', port=443): Max retries exceeded with url: /2.0/?method=chart.gettopartists&limit=50&page=62799&api_key=db2fa19152e1fab2fe1a1c48f82e705e&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025426E2D940>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))

voy a juntar los artistas en un dataframe

In [ ]:
## voy a fijarme en la estructura de una de las respuestas que he appendeado a responses
## al final, es un diccionario que tiene dentro otro diccionario y da los valores en listas
responses[0].json()

{'artists': {'artist': [{'name': 'The Weeknd',
    'playcount': '381920388',
    'listeners': '2952469',
    'mbid': 'c8b03190-306c-4120-bb0b-6f2ebfc06ea9',
    'url': 'https://www.last.fm/music/The+Weeknd',
    'streamable': '0',
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'mega'}]},
   {'name': 'Taylor Swift',
    'playcount': '1060979638',
    'listeners': '3675966',
    'mbid': '20244d07-534f-4eff-b4d4-930878889970',


In [ ]:
responses[0].json()['artists']['artist'] ## así puedo obtener la información como una lista de listas, por la que puedo iterar y convertir en dataframes

[{'name': 'The Weeknd',
  'playcount': '381920388',
  'listeners': '2952469',
  'mbid': 'c8b03190-306c-4120-bb0b-6f2ebfc06ea9',
  'url': 'https://www.last.fm/music/The+Weeknd',
  'streamable': '0',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'}]},
 {'name': 'Taylor Swift',
  'playcount': '1060979638',
  'listeners': '3675966',
  'mbid': '20244d07-534f-4eff-b4d4-930878889970',
  'url': 'https://www.last.fm/music/Taylor+Swift',
  'streama

In [ ]:
pd.DataFrame(responses[0].json()['artists']['artist'])

,name,playcount,listeners,mbid,url,streamable,image
0,The Weeknd,381920388,2952469,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Taylor Swift,1060979638,3675966,20244d07-534f-4eff-b4d4-930878889970,https://www.last.fm/music/Taylor+Swift,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
2,Kanye West,694062920,5775310,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Drake,395844249,4708104,b49b81cc-d5b7-4bdd-aadb-385df8de69a6,https://www.last.fm/music/Drake,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Kendrick Lamar,322548318,2755731,381086ea-f511-4aba-bdf9-71c753dc5077,https://www.last.fm/music/Kendrick+Lamar,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
5,Arctic Monkeys,494314877,4768047,ada7a83c-e3e1-40f1-93f9-3e73dbc9298a,https://www.last.fm/music/Arctic+Monkeys,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
6,Lana Del Rey,529682872,3110363,b7539c32-53e7-4908-bda3-81449c367da6,https://www.last.fm/music/Lana+Del+Rey,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
7,SZA,114040015,1438504,272989c8-5535-492d-a25c-9f58803e027f,https://www.last.fm/music/SZA,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
8,Ariana Grande,387191384,2438050,f4fdbb4c-e4b7-47a0-b83b-d91bbfcfa387,https://www.last.fm/music/Ariana+Grande,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
9,"Tyler, the Creator",307049868,2038088,f6beac20-5dfe-4d1f-ae02-0b0a740aafd6,"https://www.last.fm/music/Tyler,+the+Creator",0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [ ]:
pags = [pd.DataFrame(response.json()['artists']['artist']) for response in responses]
artists = pd.concat(pags)
print(artists.shape)
artists.head()

(15450, 7)


,name,playcount,listeners,mbid,url,streamable,image
0,The Weeknd,381920388,2952469,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Taylor Swift,1060979638,3675966,20244d07-534f-4eff-b4d4-930878889970,https://www.last.fm/music/Taylor+Swift,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
2,Kanye West,694062920,5775310,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Drake,395844249,4708104,b49b81cc-d5b7-4bdd-aadb-385df8de69a6,https://www.last.fm/music/Drake,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Kendrick Lamar,322548318,2755731,381086ea-f511-4aba-bdf9-71c753dc5077,https://www.last.fm/music/Kendrick+Lamar,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [ ]:
artists.info()
artists.describe().T

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15450 entries, 0 to 499
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        15450 non-null  object
 1   playcount   15450 non-null  object
 2   listeners   15450 non-null  object
 3   mbid        15450 non-null  object
 4   url         15450 non-null  object
 5   streamable  15450 non-null  object
 6   image       15450 non-null  object
dtypes: object(7)
memory usage: 965.6+ KB


,count,unique,top,freq
name,15450,10000,The Weeknd,3
playcount,15450,9995,940536,4
listeners,15450,9901,37785,6
mbid,15450,5622,,6859
url,15450,10000,https://www.last.fm/music/The+Weeknd,3
streamable,15450,1,0,15450
image,15450,2,[{'#text': 'https://lastfm.freetls.fastly.net/...,15250


In [ ]:
artists.to_csv('artists.csv')

extraigo ahora los top géneros de cada artista con `'artist.getTopTags'`

In [ ]:
def getTags(artist):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'artist.getTopTags',
        'artist':  artist
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    tags = [tag['name'] for tag in res.json()['toptags']['tag'][:5]]
    tags_str = ', '.join(tags)

In [ ]:
tqdm.pandas()

## esto es un apply pero le pongo la barra chula para que me dé el status
artists['tags'] = artists['name'].progress_apply(getTags)

100%|██████████| 15450/15450 [46:26<00:00,  5.54it/s] 


In [ ]:
artists.head()

NameError: name 'artists' is not defined

In [ ]:
artists.to_csv('artists_with_tags.csv')